In [1]:
#| default_exp differentiable_laziness
# from nbdev.showdoc import *
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
#| export
import torch
from torch import sparse
def curvature(P, diffusion_powers = 8, aperture = 20, smoothing = 1, avg_transition_probabilities = True, precomputed_powered_P = None):
  """Same as the `curvature` function, but uses pytorch as backend instead of numpy. 
  Works on both tensors and COO sparse tensors.
  """
  sample_number = 100
  row_partitions = torch.empty(sample_number)
  for idx, i in enumerate(torch.randint(P.shape[0]), size = (sample_number,)):
    row_partitions[idx] = torch.topk(P[i].dense(), aperture, largest=True, sorted = True)[0]
  P_threshold = torch.mean(row_partitions)
  # Compute powers of P
  if precomputed_powered_P is not None:
    P_powered = precomputed_powered_P
  else:
    P_powered = torch.linalg.matrix_power(P, diffusion_powers)
  near_neighbors_only = P_powered * P_threshold
  if near_neighbors_only.is_sparse:
    laziness_aggregate = sparse.sum(near_neighbors_only,dim=[1]).to_dense()
  else:
    laziness_aggregate = torch.sum(near_neighbors_only, dim=1)
  laziness = laziness_aggregate
  if smoothing:
    average_laziness = P @ laziness[:,None]
    average_laziness = average_laziness.squeeze()
    laziness = average_laziness
  return laziness


ModuleNotFoundError: No module named 'torch'

Testing:

In [ ]:
from diffusion_curvature.core import DiffusionMatrix

: 

In [ ]:
from diffusion_curvature.datasets import torus
from diffusion_curvature.core import DiffusionMatrix, plot_3d
X, ks = torus(n=5000)
P = DiffusionMatrix(X,kernel_type="adaptive",k=20)

In [ ]:
# turn P into a sparse tensor
P = torch.tensor(P).to_sparse()

In [ ]:
# calculate laziness
ks = curvature(P, diffusion_powers=8, aperture=20)
plot_3d()